# Task 2: Sentiment Analysis

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
!pip install evaluate
!pip install transformers[torch]
!pip install datasets transformers huggingface_hub

In [4]:
from datasets import load_dataset

imdb = load_dataset("imdb")

c:\Users\Darkness\miniconda3\envs\zero-shot-seg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

c:\Users\Darkness\miniconda3\envs\zero-shot-seg\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

In [8]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import numpy as np
import evaluate
 
def compute_metrics(eval_pred):
   load_accuracy = evaluate.load("accuracy")
   load_f1 = evaluate.load("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [11]:
from transformers import TrainingArguments, Trainer
 
repo_name = "fine-tuned-model"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

100%|██████████| 376/376 [03:19<00:00,  1.89it/s]

{'train_runtime': 199.4502, 'train_samples_per_second': 30.083, 'train_steps_per_second': 1.885, 'train_loss': 0.2912532928142142, 'epoch': 2.0}


TrainOutput(global_step=376, training_loss=0.2912532928142142, metrics={'train_runtime': 199.4502, 'train_samples_per_second': 30.083, 'train_steps_per_second': 1.885, 'total_flos': 782725021021056.0, 'train_loss': 0.2912532928142142, 'epoch': 2.0})

In [13]:
trainer.evaluate()

100%|██████████| 19/19 [00:06<00:00,  2.86it/s]


{'eval_loss': 0.33418965339660645,
 'eval_accuracy': 0.86,
 'eval_f1': 0.8618421052631579,
 'eval_runtime': 6.9274,
 'eval_samples_per_second': 43.306,
 'eval_steps_per_second': 2.743,
 'epoch': 2.0}

# Trying the model out

In [14]:
text_input_pos = "I love this movie! It was fantastic."
text_input_neg = "The worst movie I've ever watched. It was totally cringe. The plot was very boring. I wish I had never watched it."

# tokenize
tokens_pos = tokenizer(text_input_pos, return_tensors="pt", truncation=True, padding=True, max_length=512)
tokens_neg = tokenizer(text_input_neg, return_tensors="pt", truncation=True, padding=True, max_length=512)

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

outputs_pos = model(**tokens_pos.to(device))
outputs_neg = model(**tokens_neg.to(device))

In [22]:
predicted_class = outputs_pos.logits.argmax(dim=1).item()

In [23]:
print(f"Predicted class for positive review: {predicted_class}")

Predicted class for positive review: 1


In [24]:
logits = outputs_neg.logits
predicted_class = logits.argmax(dim=1).item()

In [25]:
print(f"Predicted class for negative review: {predicted_class}")

Predicted class for negative review: 0
